<a href="https://colab.research.google.com/github/bpratham2001/Servilia/blob/main/notebooks/fine_tune_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nn = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
#nn = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
!pip install transformers
!pip install datasets
cats_full = ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics',
        'clinical_knowledge', 'college_biology', 'college_chemistry',
        'college_computer_science', 'college_mathematics', 'college_medicine',
        'college_physics', 'computer_security', 'conceptual_physics', 'econometrics',
        'electrical_engineering', 'elementary_mathematics', 'formal_logic',
        'global_facts', 'high_school_biology', 'high_school_chemistry',
        'high_school_computer_science', 'high_school_european_history',
        'high_school_geography', 'high_school_government_and_politics',
        'high_school_macroeconomics', 'high_school_mathematics',
        'high_school_microeconomics', 'high_school_physics',
        'high_school_psychology', 'high_school_statistics',
        'high_school_us_history', 'high_school_world_history',
        'human_aging', 'human_sexuality', 'international_law',
        'jurisprudence', 'logical_fallacies', 'machine_learning',
        'management', 'marketing', 'medical_genetics', 'miscellaneous',
        'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy',
        'prehistory', 'professional_accounting', 'professional_law',
        'professional_medicine', 'professional_psychology', 'public_relations',
        'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']

cats = ['abstract_algebra', 'elementary_mathematics', 'formal_logic', 'global_facts',
        'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics',
        'high_school_psychology', 'human_aging', 'human_sexuality',
        'logical_fallacies', 'machine_learning', 'miscellaneous',
        'moral_disputes', 'moral_scenarios', 'philosophy', 'professional_psychology', 'sociology', 'world_religions']

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.6 MB/s eta 0:00:00


In [ ]:
"""
from datasets import load_dataset

d = {}
for i in range(len(cats)):
  d[cats[i]] = load_dataset('cais/mmlu', cats[i])
#d.filter(lambda x: x["subject"] in cats)
"""

In [30]:
import pandas as pd
file =('wine-tastings-guide.txt')
df1 = pd.read_csv(file, sep = ' - ')
file =('FR-IT.txt')
df2 = pd.read_csv(file, sep = ' - ')
file =('grapes.txt')
df3 = pd.read_csv(file, sep = ' - ')
file =('wine-enthusiast.txt') # Questions already framed
df4 = pd.read_csv(file, sep = ' - ')
file =('essence-of-wine.txt') # Questions already framed
df5 = pd.read_csv(file, sep = ' - ')

<ipython-input-30-510a31fafacc>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df1 = pd.read_csv(file, sep = ' - ')
<ipython-input-30-510a31fafacc>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df2 = pd.read_csv(file, sep = ' - ')
<ipython-input-30-510a31fafacc>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df3 = pd.read_csv(file, sep = ' - ')
<ipython-input-30-510a31fafacc>:9: ParserWarning: Falling bac

In [31]:
print("total data points available: " + str(df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]))

total data points available: 275


In [17]:
#target format
# how to train on df? yahma/alpaca-cleaned
sample = pd.read_json("hf://datasets/yahma/alpaca-cleaned/alpaca_data_cleaned.json")
from random import randint as r
# prompts
tastes = ["Tell me more about wines that are", "Describe to me wines that feel",
          "I would like to know more about wines considered", "I want to know more about wines that are",
          "Tell me the meaning of", "Explain, in wine terminology,", "Elaborate on wines that feel", "Enlighten me on wines that feel",
          "Educate me on", "I don't understand wines that are"]
tastes_q = ["What does it mean when a wine is", "How can a wine be", "How are wines considered", "How are wines"]
tastes_q2 = ["Why does", "How can", "Why would", "How does", "What does it mean if"]
tastes_q3 = ["feel ", "taste ", "seem "]
other = ["Tell me more about", "Describe", "I would like to know more about", "I want to know more about",
         "Tell me about", "Explain", "Elaborate on", "Enlighten me on", "Educate me on", "Recommend me ", "Give me a recommendation regarding "]
what_singular = ["What is", "What's"]

#unused
what_plural = ["What are"]
how_singular = ["How does", "How is", "How can", "How come"]
how_plural = ["How do", "How are", "How can", "How come"]
where_singular = ["Where is", "What is the location of", "Where can I find", "How do I come across"]
where_plural = ["Where are", "What are the locations of"]
why_plural = ["Why do", "Why are"]

# function to construct question/answers from blogs/articles
def construct_prompt(row, rng, q_type=0):
  if q_type == 0:#tastes
    if rng == 0:
      instruction = str(tastes[r(0, len(tastes)-1)] + " " +row['word']+".")
    elif rng == 1:
      instruction = str(tastes_q[r(0, len(tastes_q)-1)] + " " +row['word']+"?")
    else:
      instruction = str(tastes_q2[r(0, len(tastes_q2)-2)] + " a wine "+ tastes_q3[r(0,len(tastes_q3)-1)] +row['word']+"?")
  elif q_type == 1:#regions
    if rng == 0:
      instruction = str(other[r(0, len(other)-1)] + " " +row['word']+".")
    elif rng == 1:
      instruction = str("How is " +row['word']+"?")
    else:
      instruction = str(what_singular[r(0, len(what_singular)-1)] + " " +row['word']+" like?")
  else:#grapes
    if rng == 0:
      instruction = str(other[r(0, len(other)-1)] + " " +row['word']+" grapes.")
    elif rng == 1:
      instruction = str(what_singular[r(0, len(what_singular)-1)] + " " +row['word']+"?")
    else:
      instruction = str("How are the wines that are made from " +row['word']+" grapes?")
  output = row['description']
  return [instruction, output]

sample.head()

,instruction,input,output
0,Give three tips for staying healthy.,,1. Eat a balanced and nutritious diet: Make su...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,An atom is the basic building block of all mat...
3,How can we reduce air pollution?,,There are several ways to reduce air pollution...
4,Pretend you are a project manager of a constru...,,I had to make a difficult decision when I was ...


In [32]:
# formatting data into alpaca format
df0 = pd.DataFrame(columns=['instruction', 'input', 'output'])
#construct_prompt(row, rng, q_type=0)
lst = [df1, df2, df3]
for i in range(len(lst)):
  for index,row in lst[i].iterrows():
    a = construct_prompt(row, r(0,2), i)
    df0.loc[len(df0)] = [a[0],'',a[1]]
for j in [df4, df5]:
  for index,row in j.iterrows():
    df0.loc[len(df0)] = [row['question'],'',row['answer']]
  if df0.shape[0] < (df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]):
    df0 = df0.sample(frac=1)
df0 = df0.reset_index().drop('index', axis=1)
df0.iloc[50:100]

,instruction,input,output
50,What does it mean when a wine is Reglisse?,,The French term for licorice root (black licor...
51,What's Loire Valley like?,,The Loire river valley covers a large swath of...
52,How do I pair at a restaurant?,,"Don’t fret about pairing. Dining out, you’re l..."
53,How are wines Depth?,,While depth can be used to refer to the densit...
54,Wine for metallic oysters?,,While the Kimmeridgian soils of Chablis are ri...
55,Why does a wine taste Stone?,,Smelling or tasting stones in a wine is not un...
56,Describe to me wines that feel Laser-like.,,This is one of the wine descriptions used to d...
57,My pizza has clams on it.,,"The round, rich texture of oaked Chardonnay br..."
58,How can I avoid palate blowout?,,"1. Hydration, Hydration, Hydration. 2. Avoid O..."
59,How to best pair blackened salmon?,,Blackened salmon features a Cajun-inspired spi...


In [34]:
# check for NaNs in dataset
for index,row in df0.iterrows():
  if (len(str(row['output'])) < 20) or (len(str(row['instruction'])) < 10):
    print(row)
df0.shape

(275, 3)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(nn)


def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

#tokenized_datasets = ds.map(tokenize_function, batched=True) ???

KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = AutoModelForSequenceClassification.from_pretrained(nn, num_labels=4, torch_dtype="auto")
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")
#training_args = TrainingArguments(output_dir="test_trainer")
metric = evaluate.load("accuracy")

In [ ]:
for i in d:
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset= i["test"],
      eval_dataset=i["validation"],
      compute_metrics=compute_metrics)
  trainer.train()